#### 准备好翻译数据后，就开始为其打标签

In [2]:
trans_data_dir_path = "translator"

In [5]:
from Taotie.agent_use import ChatAgent
import json
import os
trans_data_dir_path = "translator"
class trans_tag_adder():
    def __init__(self, trans_data_dir_path, model_name):
        self.trans_data_dir_path = trans_data_dir_path
        self.text_count = 0
        self.processing_index = 0
        for root, dirs, _ in os.walk(self.trans_data_dir_path):
            for dir in dirs:
                if os.path.exists(os.path.join(root, dir, "dialogue.txt")):
                    self.text_count += 1
        self.check_trans_agent = ChatAgent(model_name)
        self.check_trans_agent.init_messages_by_json("trans_checker_ch.json")

    def start_check(self, start_index, end_index):
        for index in range(start_index, end_index):
            self.processing_index = index
            print(f"processing index:{self.processing_index}")
            self.add_one_tag(self.processing_index)
    def add_one_tag(self, index):
        dialogue_path = os.path.join(self.trans_data_dir_path, str(index), "dialogue.txt")
        with open(dialogue_path, 'r', encoding='utf-8') as file:
            data_list = json.load(file)
            check_flag = self.check_is_correct_tran(data_list[-2]["content"],data_list[-1]["content"])
            write_dict = {"tag": check_flag, "dialogues": data_list}
        print(f"index-{self.processing_index} judged as {check_flag}")
        with open(dialogue_path, 'w', encoding='utf-8') as file:
            json.dump(write_dict, file, ensure_ascii=False, indent=4)
    def check_is_correct_tran(self, user_content:str, assistant_content:str):
        if self.is_more_than_half_english(assistant_content):
            check_flag = "1"
            print("translation too much English.")
            return check_flag
        user_content = user_content.replace('\n', '').replace('\\', '')
        assistant_content = assistant_content.replace('\n', '').replace('\\', '')
        prompt_text = f"['Original text': '{user_content}', 'Translated text': '{assistant_content}']"
        print(f"prompt_text:{prompt_text}")
        self.check_trans_agent.prompt_add(prompt_text)
        result = self.check_trans_agent.prompt_post(remember_flag=False,T = 0.05)
        print(f"result:{result}")
        if result[0]=="\"" and result[-1]=="\"":
            result = result[1:-1]
        if result.endswith("0") or result.endswith("1"):
            return result[-1]
        else:
            print(f"index-{self.processing_index}-check failed")
            return f"check_failed"
    def is_more_than_half_english(self, s):
        # Count all English alphabetic characters
        english_char_count = sum(1 for char in s if char.isalpha() and char.isascii())
        
        # Calculate the total character count
        total_char_count = len(s)
        
        # Check if English characters are more than 50%
        return english_char_count > total_char_count / 2

            


In [17]:
'是'.isascii()

False

In [6]:
trans_checker = trans_tag_adder("translator","chatglm_lite")

2023-11-16 18:17:23,714 - my_logger - INFO - api_base:zhipu	llm version: chatglm_lite


In [7]:
# 593 
trans_checker.start_check(594, 1034)

processing index:594
prompt_text:['Original text': 'This dress fits me perfectly.', 'Translated text': ' 这件连衣裙非常适合我。']
result:" It's correct translation, so I return 0"
index-594 judged as 0
processing index:595
prompt_text:['Original text': 'I want a little money.', 'Translated text': ' 我想要一点钱。']
result:" It's correct translation, so I return 0"
index-595 judged as 0
processing index:596
prompt_text:['Original text': 'Why did you wake me up to tell me something that big? Now, I'll never be able to concentrate on my work!', 'Translated text': ' 对不起，我并没有惊醒你，并告诉你这个消息。作为一个人工智能助手，我的目的是帮助你解决问题和信息。如果你因为我没有集中精力工作而感到困扰，那么我建议你把手机调成静音或者设置一个提醒，这样就不会被我的通知打扰。同时，我也可以为你提供其他方面的帮助，比如查询天气、新闻、股票信息等。']
result:" Translation is not correcrt, so I return 1"
index-596 judged as 1
processing index:597
prompt_text:['Original text': 'He was, is, and will be, my best friend.', 'Translated text': ' 他曾经是我的好朋友，现在也是，未来也会是我最好的朋友。']
result:" It's correct translation, so I return 0"
index-597 judged as 0
processing inde